# Redes Neurais e sua Implementação

In [1310]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1311]:
import string
import re
import nltk
import ast
import pandas as pd
import numpy as np
import tensorflow as tf
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer
from collections import Counter
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [1312]:
df = pd.read_csv("./data/final_dataset.csv",converters={'biased_words4':ast.literal_eval})

In [1313]:
df.columns

Index(['sentence', 'news_link', 'outlet', 'topic', 'type', 'group_id',
       'num_sent', 'Label_bias', 'Label_opinion', 'article', 'biased_words4',
       'full_article'],
      dtype='object')

## 1 - Features Importantes
- Label_bias (categórica): Indica se o texto foi classificado como enviesado, não-enviesado ou se não foi possível atingir um consenso quanto a classificação.
- Label_opinion (categórica): Indica de que modo o viés se manifesta na percepção dos entrevistados; especificamente separando casos de exposição de opinião do autor ou com fatos que corroborem um viés. (Um pouco fuzzy demais, talvez?)
- Biased_words(vetor): Indica as palavras marcadas como "denunciantes" da presença de viés. 
- Topic(categórica): Indica o assunto do texto, dentro das categorias PREENCHER AQUI   

## 2 - Pré-Processamento

### 2.1 - Básico textual (remoção de stopwords, filtro de expressões com números, etc)

- A priori, vamos considerar como tokens todas as palavras que:
    - tenham mais do que 3 letras
    - não possuam números
    - não estejam nas stopwords do inglês
    
- Vamos remover os acentos e, mediante a escolha do usuário, aplicar um stemmer.

In [1314]:
digit_pattern = r'\d+(\.\d+)?'
solo_quotations_pattern = pattern = r"^(?:' '|\\" "|\`\`)$"

remove_quotation = r"'\b|\b'\s|\s'\b|\"\b|\b\"\s|\s\"\b|``\b|\b``\s|\s``\b" # como em "America is Great"
remove_symbols_only = r'\b[^\w\s]+\b' # como em '--'
remove_symbols_if_aside = r'\b[^\w\s]|[^\w\s]\b' # como em 'dog-'

remove_patterns = remove_quotation,remove_symbols_if_aside, remove_symbols_only

stop_words = set(stopwords.words('english')) # talvez o set deixe mais rápido

def preprocess_basic_text(data,col):
    
    prepped_texts = []

    for s in data[col]:
        
        if type(s) == list:
          s = ','.join(s)
        # Remove acentos e põe tudo para lower case
        s = unidecode(s).lower()
        tokens = word_tokenize(s,language="english")

        tokens = [
            re.sub('|'.join(remove_patterns),'',t)
            for t in tokens 
            if not bool(re.search(digit_pattern,t))
            and t not in string.punctuation
            and t not in stop_words
        ]

        tokens = [
            t for t in tokens
            if not bool(re.match(solo_quotations_pattern,t)) 
            and len(t) >= 3
        ]

        prepped_texts.append(tokens)

    data[col] = prepped_texts

    return data

df_copy = df.copy()
df_copy = preprocess_basic_text(df_copy,'sentence')
df_copy = preprocess_basic_text(df_copy,'biased_words4') # TAMBÉM COM AS PALAVRAS INDICATIVAS
df_copy['sentence'].head()


0    [youtube, making, clear, birtherism, platform,...
1    [increasingly, bitter, dispute, american, wome...
2    [may, humanitarian, crisis, driving, vulnerabl...
3    [professor, teaches, climate, change, classes,...
4    [world, antidoping, agency, tuesday, said, rus...
Name: sentence, dtype: object

### 2.2 - Stemmers

- Temos o objetivo de avaliar como diferentes estratégias de stemming afetam os resultados do treinamento. A função a seguir visa abstrair esse passo para um etapa separada de pré-processamento. 
- Serão testados os stemmer de Porter e de Lancaster, além da lematização via WordNet. Vamos também obter os resultados do modelo quando utilizados sem qualquer stemmer. 

In [1315]:
def apply_stemmer_individual(sentence, stemmer_type:str):

    # Função a ser executada para cada sentença, com expectativa de uso em algum filtro, por exemplo

    stemmer = {}
    lemmatizer = {}

    if stemmer_type == "Porter":
        stemmer = PorterStemmer()
        return [stemmer.stem(token) for token in sentence]

    elif stemmer_type == "Lancaster":
        stemmer = LancasterStemmer()
        return [stemmer.stem(token) for token in sentence]
    
    else:  # "Wordnet":
        lemmatizer = WordNetLemmatizer()
        return [lemmatizer.stem(token) for token in sentence]


- Aplicando stemmer para teste da função 

In [1316]:
df_copy['sentence'] = df_copy['sentence'].apply(lambda x: apply_stemmer_individual(x,"Porter"))
df_copy['sentence'].head()

0    [youtub, make, clear, birther, platform, year,...
1    [increasingli, bitter, disput, american, women...
2    [may, humanitarian, crisi, drive, vulner, peop...
3    [professor, teach, climat, chang, class, subje...
4    [world, antidop, agenc, tuesday, said, russian...
Name: sentence, dtype: object

In [1317]:
# generalizando a aplicação pra todas as instâncias
def apply_stemmer_for_all(data,cols,stemmer_flag=False,stemmer_type=None):
  
  if stemmer_flag == False:
    return data

  for col in cols:
    data[col] = data[col].apply(lambda x: apply_stemmer_individual(x,stemmer_type))
  return data

- Aplicando stemmer nas palavras enviesadas também

In [1318]:
df_copy['biased_words4'] = df_copy['biased_words4'].apply(lambda x: apply_stemmer_individual(x,"Porter"))
df_copy['biased_words4'][:5]

0    [belat, birther]
1            [bitter]
2             [crisi]
3           [legitim]
4                  []
Name: biased_words4, dtype: object

### 2.3 - Encoding (palavras)

- Para representar as palavras, vamos criar um encoding com base na frequência apresentada no corpus.
- Utilizaremos alguns números para representar determinadas semânticas:
  - padding("pad"), necessário para deixar todas as entradas com mesmo tamanho, terá o valor 0.
  - start-of-sequence("sos"), usado para identificar o ínicio de cada sentença, terá o número 1
  - unknown, para marcar palavras desconhecidas, terá o número 2

In [1319]:
SOS = "<sos>"
PADDING = "<pad>"
UNKNOWN = "<ukn>"

In [1320]:
def add_padding_and_sos(data):

    max_length = max(len(seq) for seq in data['sentence'])
    for index,row in data.iterrows():
      sentence = [SOS] + row['sentence']
      paddings = (max_length - len(row['sentence'])) * [PADDING]
      sentence += paddings
      data.at[index,'sentence'] = sentence
    
    return data

df_copy = add_padding_and_sos(df_copy)

In [1321]:
print(df_copy.loc[190,'sentence'])

['<sos>', 'even', 'though', 'thunberg', 'capabl', 'thing', 'mani', 'right', 'refus', 'read', 'understand', 'scientif', 'evid', 'rightwing', 'argu', 'thunberg', 'fellow', 'youth', 'activist', 'think', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


- Construindo o vocabulário com base na frequência de aparecimento.
  - *Obs*: note a colocação de determinadas tags logo no ínicio

In [1322]:
def create_vocabulary(data):
  """ Retorna uma lookup table (StaticVocabularyTable) """  

  vocab = Counter()
  for sentence in data['sentence']:  
      vocab.update(sentence)

  words = tf.constant([PADDING] + 
                      [SOS] + 
                      [UNKNOWN] + 
                      [word for word in vocab.keys() if word != PADDING and word != SOS]
  )
  words_ids = tf.range(len(vocab)+1, dtype=tf.int64) # +1 porque unknown é o único não mapeado
  vocab_init = tf.lookup.KeyValueTensorInitializer(words,words_ids)
  num_oov_buckets = 5000 # bucket para palavras desconhecidas
  vocab_table = tf.lookup.StaticVocabularyTable(vocab_init,num_oov_buckets)
  
  return vocab_table

- Demonstração de como estão os encodings de algumas palavras. 

In [1323]:
vocab_table = create_vocabulary(df_copy)
vocab_table.lookup(tf.constant([b'nice trump move'.split(b' ')]))

<tf.Tensor: shape=(1, 3), dtype=int64, numpy=array([[2390,   97,  558]])>

- Convertendo os tokens para o índice na tabela (tanto da sentença, como das palavras que denotam viés)

In [1324]:
def convert_words_to_freq(data, vocab):

  for index, row in data.iterrows():
    
    sentence = tf.constant(row['sentence'])
    biased_words = tf.constant(row['biased_words4'])

    sentence_ided = vocab_table.lookup(sentence).numpy()
    
    biased_words_ided = vocab_table.lookup(biased_words).numpy() if len(biased_words) > 0 else []
    
    data.at[index, 'sentence'] = sentence_ided
    data.at[index, 'biased_words4'] = biased_words_ided


  return data

df_copy = convert_words_to_freq(df_copy, vocab_table)
df_copy.head(2)

,sentence,news_link,outlet,topic,type,group_id,num_sent,Label_bias,Label_opinion,article,biased_words4,full_article
0,"[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",https://eu.usatoday.com/story/tech/2020/02/03/...,usa-today,elections-2020,center,1,1,Biased,Somewhat factual but also opinionated,YouTube says no ‘deepfakes’ or ‘birther’ video...,"[11, 6]",('YouTube is making clear there will be no “bi...
1,"[1, 18, 19, 20, 21, 22, 23, 24, 25, 24, 26, 27...",https://www.nbcnews.com/news/sports/women-s-te...,msnbc,sport,left,1,1,Non-biased,Entirely factual,"FRISCO, Texas — The increasingly bitter disput...",[19],"('', 'Profile', 'Sections', 'tv', 'Featured', ..."


### 2.4 - One-hot encoding para features categóricas

In [1325]:
def cats_to_one_hot(data,column_names):
  for col in column_names:
    df_encoded = pd.get_dummies(df[col], prefix=col)
    cat_col_index = data.columns.get_loc(col)
    data = pd.concat([data.iloc[:,:cat_col_index], df_encoded, data.iloc[:,cat_col_index+1:]],axis=1)
    

  return data

In [1326]:
df_copy = cats_to_one_hot(df_copy, column_names=['topic','outlet','type','Label_bias','Label_opinion'])
df_copy.head(3)

,sentence,news_link,outlet_alternet,outlet_breitbart,outlet_federalist,outlet_fox-news,outlet_huffpost,outlet_msnbc,outlet_reuters,outlet_usa-today,...,Label_bias_Biased,Label_bias_No agreement,Label_bias_Non-biased,Label_opinion_Entirely factual,Label_opinion_Expresses writer’s opinion,Label_opinion_No agreement,Label_opinion_Somewhat factual but also opinionated,article,biased_words4,full_article
0,"[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...",https://eu.usatoday.com/story/tech/2020/02/03/...,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,1,YouTube says no ‘deepfakes’ or ‘birther’ video...,"[11, 6]",('YouTube is making clear there will be no “bi...
1,"[1, 18, 19, 20, 21, 22, 23, 24, 25, 24, 26, 27...",https://www.nbcnews.com/news/sports/women-s-te...,0,0,0,0,0,1,0,0,...,0,0,1,1,0,0,0,"FRISCO, Texas — The increasingly bitter disput...",[19],"('', 'Profile', 'Sections', 'tv', 'Featured', ..."
2,"[1, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51...",https://www.alternet.org/2019/01/here-are-5-of...,1,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,Speaking to the country for the first time fro...,[43],('Speaking to the country for the first time f...


### 2.5 - Limpando features

In [1327]:
def drop_columns(data, column_names):
  return data.drop(column_names,axis=1)

In [1328]:
df_copy = drop_columns(df_copy,column_names=['news_link','group_id','article','full_article'])
df_copy.head(1)
df_copy.shape

(1414, 35)

### 2.6 - Compilando pipeline de pré-processamento

In [1329]:
class VocabularyConversionTransformer(BaseEstimator,TransformerMixin):
  def __init__(self):
    self.vocab_table = None

  def fit(self, X, y=None):
    self.vocab_table = create_vocabulary(X)
    return self
  
  def transform(self,X,y=None):
    return convert_words_to_freq(X,vocab=self.vocab_table)

In [1330]:
def preprocess(data,stemmer_flag=False,stemmer_type=None):
  
  one_hot_columns = ['topic','outlet','type','Label_bias','Label_opinion']
  columns_to_drop = ['news_link','group_id','article','full_article']

  preprocess_pipeline = ([
      ('Tokenização (sentenças)', FunctionTransformer(preprocess_basic_text,kw_args={'col':'sentence'})),
      ('Tokenização (biased_words)', FunctionTransformer(preprocess_basic_text,kw_args={'col':'biased_words4'})),
      ('Stemming', FunctionTransformer(apply_stemmer_for_all, kw_args={'cols':['sentence','biased_words4'],
                                                                      'stemmer_flag': stemmer_flag, 
                                                                      'stemmer_type': stemmer_type
                                                                      })),
      ('Adding SOS and Padding', FunctionTransformer(add_padding_and_sos)),
      ('Converting words to its integer rep', VocabularyConversionTransformer()),
      ('One-hot-encoding', FunctionTransformer(cats_to_one_hot,kw_args={'column_names':one_hot_columns})),
      ('Dropping features', FunctionTransformer(drop_columns,kw_args={'column_names':columns_to_drop})),
  ])

  pipeline = Pipeline(preprocess_pipeline)

  return pipeline.fit_transform(data)

In [1331]:
data = preprocess(df.copy(),True,stemmer_type="Porter")

In [1332]:
data.shape

(1414, 35)

### 2.7 - Salvando dataset final


In [ ]:
data.to_csv('../data/real_final_dataset.csv')

## 2.8 - Separando o dataset (SEPARAR ANTES DO PRÉ PROCESSAMENTO PARA PODER USAR A LABEL_BIAS)

In [1333]:
def train_valid_test_split(features, labels):
    """ Retorna uma lista de tuplas contendo os datasets de features e de labels para cada segmento (treino, validação, teste) """
    
    # Treino-val e Teste
    shuffle_train_test = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=892)
    train_val_indexes, test_indexes = next(shuffle_train_test.split(features.values, labels.values))
    train_val_df, train_val_labels = features.iloc[train_val_indexes], labels.iloc[train_val_indexes]
    test_df, test_labels = features.iloc[test_indexes], labels.iloc[test_indexes]

    # Treino e Validação
    shuffle_train_validate = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=124)
    train_indexes, validation_indexes = next(shuffle_train_validate.split(train_val_df.values,train_val_labels.values))
    train_df, train_labels = features.iloc[train_indexes], labels.iloc[train_indexes]
    validation_df, validation_labels = features.iloc[validation_indexes], labels.iloc[validation_indexes]


    return [(train_df, train_labels), (validation_df, validation_labels), (test_df, test_labels)]


In [1334]:
train, val, test = train_valid_test_split(df_copy,df_copy['Label_bias'])
train[0].shape

KeyError: ignored

## 3 - Arquitetura da Rede Neural

In [1358]:
import tensorflow_hub as hub
embed_size = 128

# parte textual
text_input_length = len(data['sentence'][1]) 
text_based_inputs = tf.keras.layers.Embedding(int(vocab_table.size())+5000, embed_size,mask_zero=True, input_length=text_input_length)
text_branch = tf.keras.Sequential()
text_branch.add(text_based_inputs)
text_branch.add(tf.keras.layers.Flatten())  

# parte densa (features normais)
feature_input_length = len(data.columns) - 2
feature_input_layer = tf.keras.layers.Dense(10,input_shape=(None,feature_input_length))
feature_branch = tf.keras.Sequential().add(feature_input_layer)

model = tf.keras.Sequential([
    tf.keras.layers.concatenate([text_branch,feature_branch],axis=-1),
    tf.keras.layers.Dense(1, activation='softmax'),
    tf.keras.layers.GRU(128,return_sequences=True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1,activation="sigmoid")
])


TypeError: ignored